In [42]:
import pandas as pd
from scipy.sparse import csr_matrix, csc_matrix, coo_matrix
import numpy as np

<h1> Load Data </h1>

In [2]:
movies_df = pd.read_csv('data/movies.csv')
ratings_df = pd.read_csv('data/ratings.csv')

In [3]:
original_movie_ids = set(movies_df["movieId"])
movie_id_map = {original : new for new, original in enumerate(original_movie_ids) }
movies_df["movieId"] = movies_df["movieId"].map(movie_id_map)
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
original_movie_ids = set(ratings_df["movieId"])
movie_id_map = {original : new for new, original in enumerate(original_movie_ids) }
ratings_df["movieId"] = ratings_df["movieId"].map(movie_id_map)
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


<h1> Build Raiding Matrix </h1>

In [39]:


# Get the unique user IDs and movie IDs
user_ids = ratings_df['userId'].unique()
movie_ids = ratings_df['movieId'].unique()

# Create a dictionary to map movie IDs to column indices
movie_id_map = {movie_id: i for i, movie_id in enumerate(movie_ids)}





In [65]:
len(movie_ids)

59047

In [18]:

# Initialize lists to store the row indices, column indices, and ratings
row_indices = []
col_indices = []
ratings = []

# Iterate over the ratings dataframe
for _, row in ratings_df.iterrows():
    row_indices.append(int(row['userId']))
    col_indices.append(int(row['movieId']))
    ratings.append(row['rating'])

In [102]:
# Create the sparse spatial matrix
sparse_matrix = csr_matrix((ratings, (row_indices, col_indices)))
sparse_matrix.shape

(162542, 59047)

In [34]:
coo_matrix = coo_matrix((ratings, (row_indices, col_indices)))

<h1> Define training function </h1>

In [98]:
import pandas as pd
# Calculate the column-wise average of the sparse matrix
averages = sparse_matrix.mean(axis=0)

np.shape(averages)
avg_ratings_df = pd.DataFrame(averages[0])
avg_ratings_df

    

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all().

In [96]:
first_row = sparse_matrix.getrow(0)
first_row.toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [97]:
for i in range(first_row.shape[1]):
    # Check if the column is null
    if first_row[:, i].nnz == 0:
        # Replace the null column with a value from weights
        first_row[:, i] = avg_ratings_df[i].iloc[0]
first_row.toarray()

c:\Users\miked\.virtualenvs\AverageRatingsImplementation-7JwYyYnJ\Lib\site-packages\scipy\sparse\_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


array([[1.53806401e-05, 1.37284210e+00, 4.84662426e-01, ...,
        1.32581117e-03, 1.84567681e-05, 1.01512225e-04]])

In [128]:
from sklearn.base import BaseEstimator

class AverageRatingModel(BaseEstimator):
    avg_ratings_df = None  
        
    def fit(self, training_ratings_matrix: csc_matrix):
        # Calculate the column-wise average of the training_ratings_matrix
        averages = sparse_matrix.mean(axis=0) #TODO stop including 0s in averages
        self.avg_ratings_df = pd.DataFrame(averages[0])
        
    def predict(self, user_ratings_matrix: csc_matrix):
        for i in range(first_row.shape[1]):
            # Check if the column is null
            if first_row[:, i].nnz == 0:
                # Replace the null column with a value from weights
                first_row[:, i] = self.avg_ratings_df[i].iloc[0]
        
        # Return the modified user_ratings_matrix
        return user_ratings_matrix


<h1> Run Training </h1>

In [125]:
from sklearn.model_selection import GroupShuffleSplit

# Define the groups for the split based on the value of the first column
groups = sparse_matrix[:, 0].toarray().flatten()

# Create an instance of GroupShuffleSplit
splitter = GroupShuffleSplit(n_splits=1, test_size=0.2)

train_dataset,test_dataset = next(splitter.split(X=sparse_matrix[:,1:], groups=sparse_matrix[:,0].toarray()))



In [129]:
avg_rat_model = AverageRatingModel()

avg_rat_model.fit(train_dataset)

<h1> Evaluate </h1>


In [ ]:
# predit the unseen movies agaisnt the actual ratings

#remember each TEST user should set only 80% of their ratings as seen, and 20% as unseen, random
#only seen ratings go into the fit

#Root mean-squared error (RMSE)
#Boxplot of true vs predicted ratings
#Pearson's Correlation Coefficient (R2)
#Fraction of user-movie pairs with non-zero predicted ratings
#Fraction of user-movie ratings with a predicted values (recall)
#RMSE is appropriate if we want to exactly the predict the ratings of the users.  R2 is useful if we don't care about the ranges of the ratings, just that the predicted ratings rank items in a manner consistent with the user-provided ratings.  The MovieLens ratings are in increments of 0.5.  This makes it easy to visualize the relationships between the user-provided and predicted ratings using a boxplot (user-provided ratings on the X axis, predicted ratings on the Y axis).


In [130]:
predictions = avg_rat_model.predict(test_dataset)

In [131]:
predictions

array([     0,      1,      2, ..., 162539, 162540, 162541], dtype=int64)